In [1]:
import pickle

# set the subset number to use
subset = 1

# remove index 880 from all since it was a problem data point that the visdial dataloader ignores for some reason: Check tools/functionalties.ipynb for more details

'''GoG IMPLEMENTATION'''

'''Load the GAP output of the History Graphs Batch'''
with open('../../embeddings/history/' + str(subset) + '/history_batch_GAP.pkl', 'rb') as f:
    history_batch_GAP = pickle.load(f)
    history_batch_GAP.pop(880)

'''Load the GAP output of the Question Graphs Batch'''
with open('../../embeddings/fusion/question_GAP_batch_control' + str(subset) + '.pkl', 'rb') as f:
    question_GAP_batch = pickle.load(f)
    question_GAP_batch.pop(880)

'''Load the GAP output of the Image Graphs Batch'''
with open('../../embeddings/fusion/image_GAP_batch_control' + str(subset) + '.pkl', 'rb') as f:
    image_GAP_batch = pickle.load(f)
    image_GAP_batch.pop(880)


In [2]:
# adjust the inputs to match the attn_encoder
import torch

'''Takes in a list of lists of tensors and returns a tensor of tensors: Refactoring the GAP outputs to match the attn_encoder input format'''


def adjust_inputs(inputs):
    output = []
    # flatten the list of lists
    inputs = [item for sublist in inputs for item in sublist]
    for i in inputs:
        i = i.unsqueeze(0)
        output.append(i)
    output = torch.stack(output)
    # convert torch tensor to FloatTensor
    output = output.type(torch.FloatTensor)
    return output

In [3]:
from torch.utils.data import Dataset, DataLoader

'''hi: torch.FloatTensor
The representation of history utility
Shape [batch_size x NH, T, hidden_size]'''
hi = DataLoader(adjust_inputs(history_batch_GAP), batch_size=8*10,
                shuffle=False)  # note that the batch size is 8 but shaping it to 80 since Dataloader includes the NH dimension

'''qe: torch.FloatTensor
The representation of question utility
Shape [batch_size x NH, N, hidden_size]'''
qe = DataLoader(adjust_inputs(question_GAP_batch), batch_size=8*10,
                shuffle=False)  # note that the batch size is 8 but shaping it to 80 since Dataloader includes the NH dimension

'''im: torch.FloatTensor
The representation of image utility
Shape [batch_size x NH, K, hidden_size]'''
im = DataLoader(adjust_inputs(image_GAP_batch), batch_size=8*10,
                shuffle=False)  # note that the batch size is 8 but shaping it to 80 since Dataloader includes the NH dimension

for h, q, i in zip(hi, qe, im):
    print('History Input for Attn_Encoder: ', h.shape)
    print('Question Input for Attn_Encoder: ', q.shape)
    print('Image Input for Attn_Encoder: ', i.shape)
    break


History Input for Attn_Encoder:  torch.Size([80, 1, 512])
Question Input for Attn_Encoder:  torch.Size([80, 1, 512])
Image Input for Attn_Encoder:  torch.Size([80, 1, 512])


In [4]:
import torch.nn as nn

'''mask_hi: torch.LongTensor
Shape [batch_size x NH, T]'''
history_masks = torch.ones(
    len(history_batch_GAP)*10, 1)  # 1232 different dialogs x 10 number of history rounds
mask_hi = DataLoader(history_masks, batch_size=8*10, shuffle=False)

'''mask_qe: torch.LongTensor
Shape [batch_size x NH, N]'''
question_masks = torch.ones(
    len(question_GAP_batch)*10, 1)  # 1232 different dialogs x 10 number of history rounds
mask_qe = DataLoader(question_masks, batch_size=8*10, shuffle=False)

'''mask_im: torch.LongTensor
Shape [batch_size x NH, K]'''
image_masks = torch.ones(
    len(image_GAP_batch)*10, 1)  # 1232 different dialogs x 10 number of history rounds
mask_im = DataLoader(image_masks, batch_size=8*10, shuffle=False)

for m_h, m_q, m_i in zip(mask_hi, mask_qe, mask_im):
    print('History Mask for Attn_Encoder: ', m_h.shape)
    print('Question Mask for Attn_Encoder: ', m_q.shape)
    print('Image Mask for Attn_Encoder: ', m_i.shape)
    break


History Mask for Attn_Encoder:  torch.Size([80, 1])
Question Mask for Attn_Encoder:  torch.Size([80, 1])
Image Mask for Attn_Encoder:  torch.Size([80, 1])


In [5]:
# setting the config from new_options.py
config = {'seed': 42,
          'dataset': {
              'v0.9': False,
              'overfit': False,
              'concat_hist': False,
              'max_seq_len': 20,
              'vocab_min_count': 5,
              'finetune': False,
              'is_add_boundaries': True,
              'is_return_options': True,
              'num_boxes': 'fixed',
              'glove_path': 'datasets/glove/embedding_Glove_840_300d.pkl',
              'train_feat_img_path': 'datasets/bottom-up-attention/trainval_resnet101_faster_rcnn_genome__num_boxes_100_100.h5',
              'val_feat_img_path': 'datasets/bottom-up-attention/val2018_resnet101_faster_rcnn_genome__num_boxes_100_100.h5',
              'test_feat_img_path': 'datasets/bottom-up-attention/test2018_resnet101_faster_rcnn_genome__num_boxes_100_100.h5',
              'train_json_dialog_path': 'datasets/annotations/visdial_1.0_train.json',
              'val_json_dialog_path': 'datasets/annotations/visdial_1.0_val.json',
              'test_json_dialog_path': 'datasets/annotations/visdial_1.0_test.json',
              'val_json_dense_dialog_path': 'datasets/annotations/visdial_1.0_val_dense_annotations.json',
              'train_json_word_count_path': 'datasets/annotations/visdial_1.0_word_counts_train.json'
          },
          'model': {
              'decoder_type': 'misc',
              'encoder_out': ['img', 'ques'],
              'hidden_size': 512,
              'dropout': 0.1,
              'test_mode': False,

              # image features
              'img_feat_size': 2048,
              'img_num_attns': None,
              'img_has_bboxes': False,
              'img_has_attributes': False,
              'img_has_classes': False,

              # text features
              'txt_vocab_size': 11322,
              'txt_tokenizer': 'nlp',
              'txt_bidirectional': True,
              'txt_embedding_size': 300,
              'txt_has_pos_embedding': False,
              'txt_has_layer_norm': False,
              'txt_has_decoder_layer_norm': False,

              # cross attention
              'ca_has_shared_attns': False,
              'ca_has_proj_linear': False,
              'ca_has_layer_norm': False,
              'ca_has_residual': False,
              'ca_num_attn_stacks': 1,
              'ca_num_attn_heads': 4,
              'ca_pad_size': 2,
              'ca_has_avg_attns': False,
              'ca_has_self_attns': False,
          },
          'solver': {
              # Adam optimizer
              'optimizer': 'adam',
              'adam_betas': [0.9, 0.997],
              'adam_eps': 1e-9,
              'weight_decay': 1e-5,
              'clip_norm': None,
              # dataloader
              'num_epochs': 100,
              'batch_size': 8,
              'cpu_workers': 8,
              'batch_size_multiplier': 1,
              # learning rate scheduler
              'scheduler_type': 'LinearLR',
              'init_lr': 5e-3,
              'min_lr': 1e-5,
              'num_samples': 1233,
              # warmup scheduler
              'warmup_factor': 0.2,
              'warmup_epochs': 1,
              # linear scheduler
              'linear_gama': 0.5,
              'milestone_steps': [3, 6, 8, 10, 11],
              'fp16': False,
          },
          'callbacks': {
              'resume': False,
              'validate': True,
              'path_pretrained_ckpt': None,
              'save_dir': 'checkpoints/',
              'log_dir': 'checkpoints/tensorboard/',
          }
          }


In [6]:
from visdial.encoders.attn_encoder import AttentionStack
'''The Attention Stack include of 3 blocks (i.e. 9 MHAttentions) to compute the attention from all sources to one target (including itself)
Attention from X -> Y and Y -> X can be wrapped into a single MultiHeadAttention
And self-attention X -> X: can be wrapped into MultiHeadAttention(X, X)'''

# initialize the AttentionStack
attention_stack = AttentionStack(config)
batch_output = []

for i, q, h, m_i, m_q, m_h in zip(im, qe, hi, mask_im, mask_qe, mask_hi):
    # convert to tuple
    batch_input = (i, q, h, m_i, m_q, m_h)
    # pass the inputs to the AttentionStack
    batch_output.append(attention_stack(batch_input))
    # output : A tuples of the updated representations of inputs as the triples.

In [7]:
from torch.utils.data import DataLoader
from visdial.data.dataset import VisDialDataset

train_dataset = VisDialDataset(config, 'train')

train_dataloader = DataLoader(train_dataset,
                              batch_size=config['solver']['batch_size'],
                              num_workers=config['solver']['cpu_workers'],
                              shuffle=False)


[train subset] Tokenizing questions...
[train subset] Tokenizing answers...
[train subset] Tokenizing captions...


In [8]:
import pickle
from ast import literal_eval

batch_input = {}
for batch, i, q, h, m_i, m_q, m_h in zip(train_dataloader, im, qe, hi, mask_im, mask_qe, mask_hi):
    # convert to tuple with batch['img_ids'] as the primary key
    batch_input[str(batch['img_ids'].tolist())] = (i, q, h, m_i, m_q, m_h)

# save them to disk
with open('batch_input_control.pkl', 'wb') as f:
    pickle.dump(batch_input, f)
